In [4]:
import chromadb
from chromadb.config import Settings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain.embeddings import HuggingFaceEmbeddings
from os import getcwd, environ, listdir
from pathlib import Path
from sys import path

path.append(r'C:\Users\Izogie\Desktop\Folders\Projects\Python\KB Chat\src')
from modules.SourceManager import SourceManager

In [5]:
from yaml import safe_load

def load_config(file_path="./config.yml"):
    with open(file_path, 'r') as file:
        config = safe_load(file)
        for key, value in config.items():
            environ[key] = value
load_config()

In [6]:
TST_DIR = Path(getcwd())
PROJ_DIR = TST_DIR.parent
DB_DIR = TST_DIR / "chroma_test"
DATA_DIR = PROJ_DIR / "data"

In [7]:
embedding_function = HuggingFaceEmbeddings()

c:\Python312\Lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
manager = SourceManager()
data = manager.load_json("processed_articles.jsonl")

# ParentDocumentRetriever Testing

In [ ]:
from langchain_chroma import Chroma

In [ ]:
docs = []
for file in listdir(str(DATA_DIR)):
    loader = TextLoader(str(DATA_DIR / file))
    docs.extend(loader.load())


1225


In [ ]:
vectordb = Chroma('parent_docs',
                embedding_function=embedding_function, 
                persist_directory=str(DB_DIR),
                client_settings=Settings(allow_reset=True))
vectordb.reset_collection()
print("There are", vectordb._collection.count(), "in the collection")

There are 0 in the collection


c:\Python312\Lib\site-packages\posthog\client.py:345: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().replace(tzinfo=tzutc())
c:\Python312\Lib\site-packages\posthog\client.py:345: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().replace(tzinfo=tzutc())
c:\Python312\Lib\site-packages\posthog\client.py:345: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().replace(tzinfo=tzutc())


c:\Python312\Lib\site-packages\posthog\request.py:40: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  body["sentAt"] = datetime.utcnow().replace(tzinfo=tzutc()).isoformat()


In [ ]:
child_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=100)
parent_splitter = RecursiveCharacterTextSplitter(chunk_size = 800, chunk_overlap=200)
store = InMemoryStore()
retriever = ParentDocumentRetriever(
    vectorstore=vectordb, 
    docstore=store,
    parent_splitter=parent_splitter,
    child_splitter=child_splitter)

In [ ]:
retriever.add_documents(docs)

In [ ]:
query = "Who are the shards on Roshar?"

In [ ]:
response = retriever.invoke(query)

c:\Python312\Lib\site-packages\posthog\client.py:345: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().replace(tzinfo=tzutc())


c:\Python312\Lib\site-packages\posthog\request.py:40: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  body["sentAt"] = datetime.utcnow().replace(tzinfo=tzutc()).isoformat()


In [ ]:

response
# print(response[1].page_content)

[Document(metadata={'source': 'c:\\Users\\Izogie\\Desktop\\Folders\\Projects\\Python\\KB Chat\\data\\Realmatic Theory.txt'}, page_content='Shards \n\nWhile the Shards of Adonalsium exist primarily in the Spiritual Realm, their immense power has notable effects on all three of the Realms. Perhaps most significantly, Realmatic interactions between a Shard and a planet are what define the structure of any Invested Arts associated with that Shard on that planet. Shards can consciously make some changes to this structure, but the extent to which they can interfere is limited. If a Shard used their power to create the planet in the first place (as Preservation and Ruin did with Scadrial), they will be able to exercise more control.'),
 Document(metadata={'source': 'c:\\Users\\Izogie\\Desktop\\Folders\\Projects\\Python\\KB Chat\\data\\Realmatic Theory.txt'}, page_content='While most perpendicularities are created by Shards, they can also be created by other methods. Some Invested Arts, like t

# Adding Metadata

In [ ]:
import uuid
from random import randint
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_google_genai import ChatGoogleGenerativeAI
# from langchain_community.vectorstores import Chroma
from langchain_chroma import Chroma
from langchain.retrievers.self_query.chroma import ChromaTranslator


In [ ]:
def ingest_articles(collection, articles):
    """Ingests structured article data into ChromaDB."""
    namespace_uuid = uuid.UUID('f81d4fae-7dec-11d0-a765-00a0c91e6bf6')
    documents = []
    metadatas = []
    ids = []

    for article in articles:
        if article["sections"] is None:
            continue
            # documents.append("")
            # metadatas.append({
            #     "article_title": article["title"],
            #     "paragraph_header": article["title"],
            #     "paragraph_order": 0,
            #     "links": ', '.join(article["links"]),
            # })
            # ids.append(str(uuid.uuid5(namespace_uuid, article['title'])))
        else:
            for paragraph in article["sections"]:
                documents.append(paragraph["content"])
                metadatas.append({
                    "article_title": article["title"],
                    "paragraph_header": paragraph['title'],
                    "paragraph_order": paragraph["order"],
                    "links": ', '.join(article["links"]),
                })
                ids.append(str(uuid.uuid5(namespace_uuid, f"{paragraph['title']}_{paragraph['order']}_{randint(0,10000)}")))  # Create unique IDs

    collection.add(
        documents=documents,
        metadatas=metadatas,
        ids=ids 
    )
    return collection

In [ ]:

import chromadb.utils.embedding_functions as embedding_fns
embedding_function = embedding_fns.OpenAIEmbeddingFunction(api_key=environ["OPENAI_API_KEY"])

In [ ]:
client = chromadb.PersistentClient(path=str(DB_DIR), settings=Settings(allow_reset=True))
client.reset()
collection = client.get_or_create_collection("coppermind", embedding_function=embedding_function)
ingest_articles(collection, data)  

c:\Python312\Lib\site-packages\posthog\client.py:345: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().replace(tzinfo=tzutc())
c:\Python312\Lib\site-packages\posthog\client.py:345: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().replace(tzinfo=tzutc())
c:\Python312\Lib\site-packages\posthog\request.py:40: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  body["sentAt"] = datetime.utcnow().replace(tzinfo=tzutc()).isoformat()
c:\Python312\Lib\site-packages\posthog\clien

c:\Python312\Lib\site-packages\posthog\request.py:40: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  body["sentAt"] = datetime.utcnow().replace(tzinfo=tzutc()).isoformat()


In [ ]:

from langchain_openai import OpenAIEmbeddings
vectordb = Chroma(client=client, collection_name="coppermind",
                embedding_function=OpenAIEmbeddings())

c:\Python312\Lib\site-packages\posthog\client.py:345: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().replace(tzinfo=tzutc())


c:\Python312\Lib\site-packages\posthog\request.py:40: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  body["sentAt"] = datetime.utcnow().replace(tzinfo=tzutc()).isoformat()


In [ ]:
metadata_field_info = [
    AttributeInfo(
        name= "article_title",
        description= "title of parent article",
        type= "string"
    ),
    
    AttributeInfo(
        name= "paragraph_header",
        description= "header in article_title under which content was located",
        type= "string"
    ),
    AttributeInfo(
        name= "paragraph_order",
        description= "order of paragraph under the paragraph_header",
        type= "integer",
    ),
    AttributeInfo(
        name= "links",
        description= "list of article_titles that are directly related to the parent article",
        type= "string",
    )
]
doc_content_description = "paragraph of an article from the coppermind, a knowledgebase for everything in the literary universe of the Cosmere, written by Brandon Sanderson"


In [ ]:
# llm = ChatGoogleGenerativeAI(
#     model="gemini-1.5-pro"
# )
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore=vectordb,
    document_contents=doc_content_description,
    metadata_field_info=metadata_field_info,
    structured_query_translator=ChromaTranslator()
)

In [ ]:
query = "What does gold do in allomancy?"

In [ ]:
response = retriever.invoke(query)

In [ ]:
response

[Document(metadata={'article_title': 'Allomancy', 'links': 'Odium, Kelsier, Realmatic Theory, Hoid, Knights Radiant', 'paragraph_header': 'Allomancy', 'paragraph_order': 1}, page_content='Allomancy is the most widely used form of Investiture on Scadrial, and is locally known as one of the three Metallic Arts. People who have one or more Allomantic abilities are called Allomancers with each Allomantic power being paired with a type of metal, which must be ingested and "burned" to activate.'),
 Document(metadata={'article_title': 'Allomancy', 'links': 'Odium, Kelsier, Realmatic Theory, Hoid, Knights Radiant', 'paragraph_header': 'History', 'paragraph_order': 2}, page_content="Prior to the Ascension of the Lord Ruler, Allomancy was rarely found among the general population. The Lord Ruler's Ascension, and his subsequent distribution of the lerasium beads, increased the strength of the Allomantic spiritual DNA in the Final Empire, which had long lasting effects through the subsequent gener

# Keyword augmentation

In [6]:
from google.generativeai import GenerativeModel, configure
configure(api_key=environ["GOOGLE_API_KEY"])

In [15]:
from ratelimit import limits, RateLimitException, sleep_and_retry
from backoff import on_exception, expo
import google.api_core.exceptions as google_exceptions
from tqdm import tqdm
import chromadb
from chromadb.config import Settings

In [8]:
import uuid
from random import randint
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_google_genai import ChatGoogleGenerativeAI
# from langchain_community.vectorstores import Chroma
from langchain_chroma import Chroma
from langchain.retrievers.self_query.chroma import ChromaTranslator


In [9]:
MINUTE = 60
# rate is 1 QPS.
@sleep_and_retry # If there are more request to this function than rate, sleep shortly
@on_exception(expo, google_exceptions.ResourceExhausted, max_tries=10) # if we receive exceptions from Google API, retry
@limits(calls=40, period=MINUTE)
def call_prompt_in_rate(prompt):
  """
  Calls the LLM model and applies rate limits.
  """
  # return model.invoke(prompt) #Langchain openai
  return model.generate_content(prompt)


In [10]:
def ingest_articles(collection, model, articles):
    """Ingests structured article data into ChromaDB."""
    namespace_uuid = uuid.UUID('f81d4fae-7dec-11d0-a765-00a0c91e6bf6')
    documents = []
    metadatas = []
    ids = []
    with tqdm(len(articles)) as pbar_art:
        for i, article in enumerate(articles):
            if article["sections"] is None:
                continue
            else:
                with tqdm(len(article['sections'])) as pbar_para:
                    for j, paragraph in enumerate(article["sections"]):
                        content = paragraph['content']
                        prompt = f"""
                            For the following paragraph, extract a list of keywords that will be used as metadata when the paragraph is stored in a vector database. The keywords will be used to help return accurate results when the database is queried. the list must look like this "keyword1, keyword 2, keyword 3, ..."

                            Here is the paragraph:
                            ```
                            {content}
                            ```
                            """
                        # response = model.generate(prompt)
                        response = call_prompt_in_rate(prompt)
                        try:
                            keywords = response.text
                            # keywords = response.content
                        except Exception as e:
                            print(e)
                            print(f"{i}, {j}", content)
                            keywords=""
                            continue
                        documents.append(paragraph["content"])
                        metadatas.append({
                            "article_title": article["title"],
                            "paragraph_header": paragraph['title'],
                            "paragraph_order": paragraph["order"],
                            "links": ', '.join(article["links"]),
                            "keywords": keywords,
                        })
                        ids.append(str(uuid.uuid5(namespace_uuid, f"{paragraph['title']}_{paragraph['order']}_{randint(0,10000)}")))  # Create unique IDs
                        pbar_para.update(1)
            pbar_art.update(1)

    collection.add(
        documents=documents,
        metadatas=metadatas,
        ids=ids 
    )
    return collection

In [11]:
model = GenerativeModel(model_name="gemini-1.5-flash")

In [12]:
import chromadb.utils.embedding_functions as embedding_fns
embedding_function = embedding_fns.OpenAIEmbeddingFunction(api_key=environ["OPENAI_API_KEY"])

In [13]:
client = chromadb.PersistentClient(path=str(DB_DIR), settings=Settings(allow_reset=True))
# client.reset()
collection = client.get_or_create_collection("coppermind", embedding_function=embedding_function)
# response = ingest_articles(collection, model, data)

c:\Python312\Lib\site-packages\posthog\client.py:345: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().replace(tzinfo=tzutc())
c:\Python312\Lib\site-packages\posthog\client.py:345: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().replace(tzinfo=tzutc())


c:\Python312\Lib\site-packages\posthog\request.py:40: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  body["sentAt"] = datetime.utcnow().replace(tzinfo=tzutc()).isoformat()


In [16]:
from langchain_openai import OpenAIEmbeddings
vectordb = Chroma(client=client, collection_name="coppermind",embedding_function=OpenAIEmbeddings())

c:\Python312\Lib\site-packages\posthog\client.py:345: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().replace(tzinfo=tzutc())


c:\Python312\Lib\site-packages\posthog\request.py:40: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  body["sentAt"] = datetime.utcnow().replace(tzinfo=tzutc()).isoformat()


In [14]:
metadata_field_info = [
    AttributeInfo(
        name= "article_title",
        description= "title of parent article",
        type= "string"
    ),
    AttributeInfo(
        name= "paragraph_header",
        description= "header in article_title under which content was located",
        type= "string"
    ),
    AttributeInfo(
        name= "paragraph_order",
        description= "order of paragraph under the paragraph_header",
        type= "integer",
    ),
    AttributeInfo(
        name= "links",
        description= "list of article_titles that are directly related to the parent article",
        type= "string",
    ),
    AttributeInfo(
        name= "keywords",
        description= "keywords found in the article for use in queries",
        type= "string"
    ),
]
doc_content_description = "paragraph of an article from the coppermind, a knowledgebase for everything in the literary universe of the Cosmere, written by Brandon Sanderson"


In [57]:
# llm = ChatGoogleGenerativeAI(
#     model="gemini-1.5-pro"
# )
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore=vectordb,
    document_contents=doc_content_description,
    metadata_field_info=metadata_field_info,
    search_kwargs={"k": 10},
    structured_query_translator=ChromaTranslator(), 
    enable_limit=True,
    verbose=True
)

In [71]:
query = "What are Hoid's powers?"

In [72]:
response = retriever.invoke(query)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:langchain.retrievers.self_query.base:Generated Query: query='Hoid powers' filter=None limit=None
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [73]:
response

[Document(metadata={'article_title': 'Cosmere', 'keywords': 'Hoid, worldhopper, Shattering of Adonalsium, Invested items, Invested Arts, Odium, cosmere, Frost, Endowment, Autonomy, Harmony, Fortune \n', 'links': "Stormfather, Kelsier, Cognitive Shadow, Dalinar Kholin, Pits of Hathsin, Shadesmar, Odium, Honor's Perpendicularity, Awakening, Tanavast, Hoid, Urithiru, Fused, Realmatic Theory, Allomancy, Knights Radiant", 'paragraph_header': 'Hoid', 'paragraph_order': 1}, page_content="Hoid is a mysterious worldhopper who has been alive since before the Shattering of Adonalsium and was present for that event. He has collected a variety of Invested items from around the cosmere and gained access to multiple Invested Arts. His long-term goals are unclear, but he is an adamant opponent of Odium's attempts to become the sole god of the cosmere. He has written to Frost, Endowment, Autonomy, and Harmony, hoping to enlist their help in dealing with Odium, though only Harmony has responded positive

## Reranking

In [67]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank

In [17]:
import uuid
from random import randint
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_google_genai import ChatGoogleGenerativeAI
# from langchain_community.vectorstores import Chroma
from langchain_chroma import Chroma
from langchain.retrievers.self_query.chroma import ChromaTranslator

In [68]:
compressor = FlashrankRerank()
rerank_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=retriever)

In [74]:
ranked_docs = rerank_retriever.invoke(query)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:langchain.retrievers.self_query.base:Generated Query: query='Hoid powers' filter=None limit=None
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [75]:
ranked_docs

[Document(metadata={'id': 3, 'relevance_score': 0.99919415}, page_content="Hoid does not yet have the abilities of every planet he's visited. Some of his powers predate the Shattering, but none of them are gained through Hemalurgy. Hoid is concerned about opening himself up to the influence of Shards, so he only gains access to manifestations of Investiture through more conventional, but more difficult ways. Hoid has access to multiple Invested Arts, but not an incredibly large number. He is likely the most powerful non-Shard entity in the cosmere, although a few of the older dragons and aethers might rival him."),
 Document(metadata={'id': 5, 'relevance_score': 0.99868804}, page_content='Hoid is a worldhopper and Dawnshard that travels the cosmere, originally born on the planet Yolen before the Shattering. He was one of the major figures in the Shattering of Adonalsium, but avoided taking up a Shard himself, instead choosing to wander the cosmere pursing his unknown goals, often in th

# RAG Testing

In [42]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank
from langchain_google_genai import GoogleGenerativeAI

In [11]:
import chromadb.utils.embedding_functions as embedding_fns
embedding_function = embedding_fns.OpenAIEmbeddingFunction(api_key=environ["OPENAI_API_KEY"])

In [12]:
client = chromadb.PersistentClient(path=str(DB_DIR), settings=Settings(allow_reset=True))
# client.reset()
collection = client.get_or_create_collection("coppermind", embedding_function=embedding_function)
# response = ingest_articles(collection, model, data)

c:\Python312\Lib\site-packages\posthog\client.py:345: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().replace(tzinfo=tzutc())
c:\Python312\Lib\site-packages\posthog\client.py:345: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().replace(tzinfo=tzutc())


c:\Python312\Lib\site-packages\posthog\request.py:40: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  body["sentAt"] = datetime.utcnow().replace(tzinfo=tzutc()).isoformat()


In [43]:
from langchain_openai import ChatOpenAI
from google.generativeai import GenerativeModel, configure
configure(api_key=environ["GOOGLE_API_KEY"])


chatgpt = ChatOpenAI(temperature=0)
gemini = GoogleGenerativeAI(model="gemini-1.5-flash")
# gemini = GenerativeModel(model_name="gemini-1.5-flash")

In [44]:
from langchain_openai import OpenAIEmbeddings
vectordb = Chroma(client=client, collection_name="coppermind",embedding_function=OpenAIEmbeddings())

c:\Python312\Lib\site-packages\posthog\client.py:345: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().replace(tzinfo=tzutc())


c:\Python312\Lib\site-packages\posthog\request.py:40: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  body["sentAt"] = datetime.utcnow().replace(tzinfo=tzutc()).isoformat()


In [19]:
metadata_field_info = [
    AttributeInfo(
        name= "article_title",
        description= "title of parent article",
        type= "string"
    ),
    AttributeInfo(
        name= "paragraph_header",
        description= "header in article_title under which content was located",
        type= "string"
    ),
    AttributeInfo(
        name= "paragraph_order",
        description= "order of paragraph under the paragraph_header",
        type= "integer",
    ),
    AttributeInfo(
        name= "links",
        description= "list of article_titles that are directly related to the parent article",
        type= "string",
    ),
    AttributeInfo(
        name= "keywords",
        description= "keywords found in the article for use in queries",
        type= "string"
    ),
]
doc_content_description = "paragraph of an article from the coppermind, a knowledgebase for everything in the literary universe of the Cosmere, written by Brandon Sanderson"


In [45]:
# llm = ChatGoogleGenerativeAI(
#     model="gemini-1.5-pro"
# )
gpt_retriever = SelfQueryRetriever.from_llm(
    chatgpt,
    vectorstore=vectordb,
    document_contents=doc_content_description,
    metadata_field_info=metadata_field_info,
    search_kwargs={"k": 10},
    structured_query_translator=ChromaTranslator(), 
    enable_limit=True,
    verbose=True
)
gemini_retriever  = SelfQueryRetriever.from_llm(
    gemini,
    vectorstore=vectordb,
    document_contents=doc_content_description,
    metadata_field_info=metadata_field_info,
    search_kwargs={"k": 10},
    structured_query_translator=ChromaTranslator(), 
    enable_limit=True,
    verbose=True
)

In [61]:
response = gemini_retriever.invoke(query)
targ_doc = response[3]
targ_doc

INFO:langchain.retrievers.self_query.base:Generated Query: query='Dalinar Rift' filter=None limit=None
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Document(metadata={'article_title': 'Dalinar Kholin', 'keywords': 'Dalinar, memories, Rift, drinking, screams, Evi, Rathalas, burning, dependence, alcohol, family, strangers \n', 'links': "Stormfather, Surgebinder, Shadesmar, Odium, Honor's Perpendicularity, Kaladin, Hoid, Urithiru, Fused, Cosmere, Knights Radiant", 'paragraph_header': 'The Rift', 'paragraph_order': 8}, page_content="Years later, Dalinar's reaction to his memories of what happened at the Rift was to drink. Immediately after and even years later, Dalinar can hear the screams of Evi and the people of Rathalas, burning. Dalinar's dependence on alcohol was very serious, to the point that his family sometimes hid his alcohol from him, and he was reduced to asking it from strangers.")

In [62]:
targ_doc.metadata['keywords']


'Dalinar, memories, Rift, drinking, screams, Evi, Rathalas, burning, dependence, alcohol, family, strangers \n'

In [46]:
compressor = FlashrankRerank()
gpt_ranker = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=gpt_retriever)
gemini_ranker = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=gemini_retriever)

In [47]:
query = "What happened to Dalinar at the Rift"

In [48]:
gpt_docs = gpt_ranker.invoke(query)
gemini_docs = gemini_ranker.invoke(query)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:langchain.retrievers.self_query.base:Generated Query: query='Dalinar Rift' filter=None limit=None
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:langchain.retrievers.self_query.base:Generated Query: query='Dalinar Rift' filter=None limit=None
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [49]:
gemini_docs

[Document(metadata={'id': 3, 'relevance_score': 0.99934375}, page_content="Years later, Dalinar's reaction to his memories of what happened at the Rift was to drink. Immediately after and even years later, Dalinar can hear the screams of Evi and the people of Rathalas, burning. Dalinar's dependence on alcohol was very serious, to the point that his family sometimes hid his alcohol from him, and he was reduced to asking it from strangers."),
 Document(metadata={'id': 1, 'relevance_score': 0.9988333}, page_content='Dalinar was raised in the Vorin religion; his Calling is to be a leader, and his chosen Glory is determination. At some point in his life, he joined a lay devotary called the Order of Talenelat. However, over time his faith wavers, and his visions lead him to conclude that the Almighty is dead. This ends with his excommunication from the Vorin church.'),
 Document(metadata={'id': 9, 'relevance_score': 0.9982626}, page_content='Dalinar has a Nahel bond with the Stormfather, the

In [50]:
gpt_docs

[Document(metadata={'id': 3, 'relevance_score': 0.99934375}, page_content="Years later, Dalinar's reaction to his memories of what happened at the Rift was to drink. Immediately after and even years later, Dalinar can hear the screams of Evi and the people of Rathalas, burning. Dalinar's dependence on alcohol was very serious, to the point that his family sometimes hid his alcohol from him, and he was reduced to asking it from strangers."),
 Document(metadata={'id': 1, 'relevance_score': 0.9988333}, page_content='Dalinar was raised in the Vorin religion; his Calling is to be a leader, and his chosen Glory is determination. At some point in his life, he joined a lay devotary called the Order of Talenelat. However, over time his faith wavers, and his visions lead him to conclude that the Almighty is dead. This ends with his excommunication from the Vorin church.'),
 Document(metadata={'id': 9, 'relevance_score': 0.9982626}, page_content='Dalinar has a Nahel bond with the Stormfather, the

In [51]:

prompt = f"""
Use the below context to assist in answering this question: {query}

context
```
{gpt_docs}
```
"""
gpt_response = chatgpt.invoke(prompt)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [52]:

prompt = f"""
Use the below context to assist in answering this question: {query}

context
```
{gemini_docs}
```
"""
gem_response = gemini.invoke(prompt)

In [57]:
print(gpt_response.content)

At the Rift, Dalinar experienced a traumatic event that haunted him for years. He heard the screams of his wife Evi and the people of Rathalas burning, leading him to turn to alcohol as a coping mechanism. His dependence on alcohol became so serious that his family had to hide it from him, and he even resorted to asking strangers for it. This event had a profound impact on Dalinar's mental and emotional well-being.


In [59]:
print(gem_response)

The provided context doesn't contain information about what specifically happened to Dalinar at the Rift.  

We know that:

* **Dalinar has traumatic memories from the Rift:** He hears the screams of Evi and the people of Rathalas burning, which leads to his severe alcoholism.
* **Dalinar's faith is shaken:** He concludes that the Almighty is dead, leading to his excommunication from the Vorin church.

However, the details of the events at the Rift itself are not mentioned in the context. 

To understand what happened to Dalinar at the Rift, you would need to consult other sources like the books or other relevant materials. 

